In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
import datetime
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants
import random
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-08-28 15:33:45.905470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-28 15:33:46.505099: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60)

In [6]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [7]:
train_and_evaluate(model, train_data, val_data, test_data, epochs=20)

Epoch 1/20
591/591 [==============================] - 12s 17ms/step - loss: 1.3614 - accuracy: 0.2878 - val_loss: 1.3457 - val_accuracy: 0.2888
Epoch 2/20
591/591 [==============================] - 10s 17ms/step - loss: 1.3350 - accuracy: 0.3543 - val_loss: 1.2802 - val_accuracy: 0.4082
Epoch 3/20
591/591 [==============================] - 10s 17ms/step - loss: 1.2642 - accuracy: 0.4062 - val_loss: 1.2002 - val_accuracy: 0.4224
Epoch 4/20
591/591 [==============================] - 10s 17ms/step - loss: 1.1865 - accuracy: 0.4539 - val_loss: 1.0777 - val_accuracy: 0.5520
Epoch 5/20
591/591 [==============================] - 10s 17ms/step - loss: 1.0856 - accuracy: 0.5207 - val_loss: 0.9414 - val_accuracy: 0.6036
Epoch 6/20
591/591 [==============================] - 10s 17ms/step - loss: 0.9816 - accuracy: 0.5828 - val_loss: 0.8531 - val_accuracy: 0.6671
Epoch 7/20
591/591 [==============================] - 10s 17ms/step - loss: 0.9104 - accuracy: 0.6184 - val_loss: 0.7811 - val_accuracy:

In [42]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model